In [1]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec


from basics import *
from cnn_config import *
from cnn_enc_dec import *
%matplotlib inline

translating es to en
callhome es-en word level configuration
vocab size, en=5691, fr=8904
CNN - based on Lee et. al.
callhome es-en word level configuration
vocab size, en=51, fr=51


In [2]:
xp = cuda.cupy if gpuid >= 0 else np

In [3]:
text_data = pickle.load(open(text_data_dict, "rb"))

In [ ]:
model = SpeechEncoderDecoder(SPEECH_DIM, vocab_size_en, num_layers_enc, num_layers_dec,
                               hidden_units, gpuid, attn=use_attn)
if gpuid >= 0:
    cuda.get_device(gpuid).use()
    model.to_gpu()

In [ ]:
log_train_fil_name, text_fname, dev_fname, test_fname

In [ ]:
%aimport nmt_trials
from nmt_trials import *

In [ ]:
buckets, bucket_lengths = populate_buckets()

In [ ]:
25*32, 24*35

In [ ]:
_, en, sf = get_data_item('053.181')
xp.isnan(xp.sum(sf))

In [ ]:
buck_indx = 23
i = 0
batch_size = 50

pad_size_speech = (buck_indx+1) * SPEECH_BUCKET_WIDTH
pad_size_en = min(bucket_lengths[buck_indx][3], MAX_EN_LEN)

src_lim = pad_size_speech
tar_lim = pad_size_en

sp_files_in_batch = [t[0] for t in buckets[buck_indx][i:i+batch_size]]

# get the next batch of data
batch_data = []
for sp_fil in sp_files_in_batch:
    _, en_ids, speech_feat = get_data_item(sp_fil, cat="train")
    # print(speech_feat.shape, len(en_ids))
    batch_data.append((speech_feat[:pad_size_speech], en_ids[:pad_size_en]))

# compute loss
# loss = model.encode_decode_train_batch(batch_data, pad_size_speech, pad_size_en, train=True)

In [ ]:
batch_size = len(batch_data)
in_shape_r, in_shape_c = batch_data[0][0].shape

fwd_encoder_batch = xp.full((batch_size, pad_size_speech, in_shape_c), PAD_ID, dtype=xp.float32)
rev_encoder_batch = xp.full((batch_size, pad_size_speech, in_shape_c), PAD_ID, dtype=xp.float32)
decoder_batch = xp.full((batch_size, pad_size_en+2), PAD_ID, dtype=xp.int32)

In [ ]:
for i, (src, tar) in enumerate(batch_data):
    fwd_encoder_batch[i] = model.pad_array(src, src_lim)
    rev_encoder_batch[i] = model.pad_array(xp.flip(src, axis=0), src_lim)

    tar_data = [GO_ID] + tar + [EOS_ID]
    decoder_batch[i] = model.pad_list(tar_data, tar_lim+2, at_start=False)

In [ ]:
train=True

In [ ]:
fwd_encoder_batch = xp.swapaxes(fwd_encoder_batch, 0,1)
rev_encoder_batch = xp.swapaxes(rev_encoder_batch, 0,1)
decoder_batch = xp.swapaxes(decoder_batch, 0,1)

In [ ]:
L_FWD_STATES = model.encode_speech_batch_lstm(fwd_encoder_batch, model.lstm_enc, train=True)

In [ ]:
L_REV_STATES = model.encode_speech_batch_lstm(rev_encoder_batch, model.lstm_rev_enc, train)
# reverse the states to align them with forward encoder
L_REV_STATES = xp.flip(L_REV_STATES, axis=0)

In [ ]:
return_shape = L_FWD_STATES.shape
model.enc_states = F.concat((L_FWD_STATES, L_REV_STATES), axis=2)
model.enc_states = F.swapaxes(model.enc_states, 0, 1)

In [ ]:
model.encode_decode_train_batch(batch_data, pad_size_speech, pad_size_en, train=train)

In [ ]:
L_FWD_STATES.shape

In [ ]:
pad_size_speech / 8

In [ ]:
in_size, batch_size, in_dim = fwd_encoder_batch.shape

In [ ]:
print(in_size, batch_size, in_dim)

In [4]:
speech_feats = xp.load(os.path.join(speech_dir, "train.npz"))

In [5]:
haha = speech_feats["041.001"]

In [6]:
haha.shape

(1167, 39)

In [14]:
haha = xp.array([[2,-2], [1,-1], [3,-3]], dtype=xp.float32)

In [15]:
haha

array([[ 2., -2.],
       [ 1., -1.],
       [ 3., -3.]], dtype=float32)

In [16]:
meh = F.expand_dims(haha.swapaxes(0,1), axis=0)

In [17]:
meh.shape

(1, 2, 3)

In [18]:
def cnn_out_size(i, p, k):
    o = (i-k) + 2*p + 1
    print("cnn out = {0:d}".format(o))

In [19]:
k_width = 2
pad = k_width // 2
# pad = k_width - 1
pad = 1

In [178]:
if gpuid >= 0:
    # print("here")
    cuda.get_device(gpuid).use()
c1 = L.ConvolutionND(ndim=1, in_channels=SPEECH_DIM, out_channels=1, ksize=k_width, pad=pad).to_gpu()

In [179]:
c1.W.shape

(1, 39, 2)

In [180]:
i = meh.shape[2]
out_shape = (i-k_width) + 2*pad + 1
print(i, out_shape)

1167 1168


In [147]:
%%timeit -n1 -r2
ha = c1(meh)
print(ha.shape)
print(ha.data)

(1, 1, 1168)
[[[-0.26876935  0.13223523  0.33726585 ...,  0.58413428  0.69508243
    0.65751576]]]
(1, 1, 1168)
[[[-0.26876935  0.13223523  0.33726585 ...,  0.58413428  0.69508243
    0.65751576]]]
1 loop, best of 2: 2.34 ms per loop


### test with small arrays

In [24]:
mini_test = xp.array([[2,-2], [1,-1], [3,-3]], dtype=xp.float32)

In [81]:
mini_test

array([[ 2., -2.],
       [ 1., -1.],
       [ 3., -3.]], dtype=float32)

In [82]:
mini_var = F.expand_dims(haha.swapaxes(0,1), axis=0)

In [83]:
mini_var.shape

(1, 2, 3)

In [148]:
mini_k = 2; mini_pad = mini_k // 2

if gpuid >= 0:
    # print("here")
    cuda.get_device(gpuid).use()
minic1 = L.ConvolutionND(ndim=1, in_channels=2, out_channels=1, ksize=mini_k, stride=1, pad=mini_pad).to_gpu()

In [149]:
cnn_out_size(i=3, k=mini_k, p=mini_pad)

cnn out = 4


In [150]:
print(minic1.W.shape)
print(minic1.W.data)

(1, 2, 2)
[[[-0.0521806   0.37492761]
  [ 0.11593072  0.37766585]]]


In [151]:
one = minic1.W.data[0]
one

array([[-0.0521806 ,  0.37492761],
       [ 0.11593072,  0.37766585]], dtype=float32)

In [152]:
mini_test[0], mini_test[1]

(array([ 2., -2.], dtype=float32), array([ 1., -1.], dtype=float32))

In [153]:
h1 = one[0][0]*mini_test[0][0]
h2 = one[1][0]*mini_test[0][1]
h3 = one[0][1]*mini_test[1][0]
h4 = one[1][1]*mini_test[1][1]

In [154]:
sum([h1, h2, h3, h4])

array(-0.3389608860015869, dtype=float32)

In [155]:
mini_cnn_out = minic1(mini_var).data

In [156]:
mini_cnn_out.shape, mini_cnn_out

((1, 1, 4),
 array([[[-0.00547647, -0.33896089, -0.17632601, -0.50433397]]], dtype=float32))

In [158]:
F.max_pooling_nd(mini_cnn_out, ksize=2, stride=1, pad=0).data

/afs/inf.ed.ac.uk/user/s14/s1444673/anaconda3/envs/mtenv/lib/python3.6/site-packages/chainer/utils/experimental.py:104: FutureWarning: chainer.functions.pooling.MaxPoolingND is experimental. The interface can change in the future.
  FutureWarning)


array([[[-0.00547647, -0.17632601, -0.17632601]]], dtype=float32)